# Imports

In [ ]:
%load_ext autoreload

In [ ]:
import os, sys
import holoviews as hv
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))
import hpc05

import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)
def assert_cluster_checked():
    assert run_cluster.value is True, "Command not run. Check the box above to run."

In [ ]:
import adaptive
adaptive.notebook_extension()

import numpy as np
import scipy.constants
import cmath

import functools as ft

import sns_system, plotting_results
from distributed_sns import AggregatesSimulationSet as ASS
from distributed_sns import SimulationSet as SS

### Define constants

In [ ]:
constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Cluster setup

In [ ]:
run_cluster

In [ ]:
assert_cluster_checked()
hpc05.kill_remote_ipcluster()
run_cluster.value = False

In [ ]:
assert_cluster_checked()
client, dview, lview = hpc05.start_remote_and_connect(100, folder='~/two_dim_majoranas', timeout=180)
run_cluster.value = False

# Define and plot system

In [ ]:
syst_pars = {'L_down' :  1000,
               'L_up' : 1000,
               'Lm' : 250,
               'Lx' : 12.5,
               'a' :  12.5,
            'mu_from_bottom_of_spin_orbit_bands': True}

_=plotting_results.plot_syst(syst_pars, sns_system.dummy_params)

# Define standard parameters

In [ ]:
params_raw= dict(g_factor_middle = 10,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 20.0,
                 alpha_middle = 28,
                 alpha_left = 28,
                 alpha_right = 28,
                 Delta_left = .18,
                 Delta_right = .18,
                 B = 0.5,
                 phase = np.pi/2,
                 T = 0.025,
                 V=0)

params = dict(**constants,
              **params_raw)

### Define keys to be varied

In [ ]:
keys_with_bounds = {"phase":[0, 2*np.pi],
                    "B":[0,2.0]}

### Define metrics to be recorded

In [ ]:
metric_params_dict = { "pfaffian":{},
#                        "current":{"tol":1e-6},
                       "transparency":{},
#                        "energy_gap":{}
                     }

### Define data folder

In [ ]:
data_folder = 'junction_width'

# Make ASS

In [ ]:
ass = ASS(keys_with_bounds,
              syst_pars, params,
              metric_params_dict)

### Add additional dimensions

### Temperature

In [ ]:
temperatures = [0.0, 0.1, 0.2, 0.3]

def _params_temperature(syst_pars, params, temperature):
    for key in ['T']:
        if key in params:
            params[key] = temperature
        if key in syst_pars:
            syst_pars[key] = temperature
    return temperature

params_temperature = [ft.partial(_params_temperature,
                           temperature=_temperature
                           ) for _temperature in temperatures]

ass.add_dimension("temperature", params_temperature)

### Widths

In [ ]:
widths = [250, 1000]#[150, 250, 500, 1000]

def _params_width(syst_pars, params, width):
    for key in ['Lm']:
        if key in params:
            params[key] = width
        if key in syst_pars:
            syst_pars[key] = width
    return width

params_width = [ft.partial(_params_width,
                           width=_width
                           ) for _width in widths]

ass.add_dimension("width", params_width)

### Potential

In [ ]:
Vs = [0, 5, 10]

def _params_V(syst_pars, params, V):
    for key in ['V']:
        if key in params:
            params[key] = V
        if key in syst_pars:
            syst_pars[key] = V
    return V

params_V = [ft.partial(_params_V,
                           V=_V
                           ) for _V in Vs]

ass.add_dimension("V", params_V)

### Make learner

In [ ]:
ass.make_balancing_learner(1000)
ass.load(data_folder, 1000)

### Make runner with saver

In [ ]:
runner = adaptive.Runner(ass.get_balancing_learner(), executor=client)
ass.start_periodic_saver(runner, data_folder, interval=180)
runner.live_info(update_interval=1)

In [ ]:
runner.cancel()

# Plot

In [ ]:
ass.learner._points.clear()
ass.learner._loss.clear()

In [ ]:
{k:(v[1][0],ass.learner._loss.get(k)) for k,v in ass.learner._points.items()}

In [ ]:
sum([l.npoints for l in ass.learners])

In [ ]:
[(l.npoints, l.loss()) for l in ass.learner.learners]

In [ ]:
N_POINTS = 400
hv.extension('matplotlib')
kdims, plot_dict = ass.get_plot_dict(N_POINTS, tables=False, contour_pfaffian=True)
for k,v in plot_dict.items():
    plot_dict[k] = v.opts({'Image': dict(plot={'aspect':1}, norm={'axiswise':True, 'framewise':True})})

In [ ]:
kdims, plot_dict = ass.get_plot_dict(200, tables=False, contour_pfaffian=False)

In [ ]:
%%opts Image {+axiswise} [aspect=1 colorbar=True]
hv.HoloMap(plot_dict, kdims=kdims)

### Plot fermi surface

In [ ]:
kf_no_soi = lambda mu: np.sqrt(2*params['m_eff']*mu)/params['hbar']
kfso = lambda alpha: params['m_eff']*alpha/params['hbar']**2
def plot_fermi_surface(mu, alpha, **_):
    theta = np.linspace(0, 2*np.pi)
    r0 = kf_no_soi(mu) - kfso(alpha)
    r1 = kf_no_soi(mu) + kfso(alpha)
    x0, y0 = (r0*np.cos(theta), r0*np.sin(theta))
    x1, y1 = (r1*np.cos(theta), r1*np.sin(theta))
    return (hv.Path((x0,y0))*hv.Path((x1,y1)))[-r1:r1,-r1:r1]

In [ ]:
plot_dict_soi_surface = {}
for k,v in plot_dict.items():
    plot_dict_soi_surface[k] = plot_fermi_surface(**dict(zip(kdims, k)))

In [ ]:
hv.HoloMap(plot_dict_soi_surface, kdims=kdims)

### Plot line through minimum current and make critical current plot

In [ ]:
plot_dict_Ic = {}
for k,v in plot_dict.items():
    current_plot = plot_dict[(k[0], k[1], k[2], 'current')]
    image =current_plot.items()[0][1]
    
    x0, y0, x1, y1 = image.lbrt
    xdim=np.linspace(x0, x1, N_POINTS)
    ydim=np.linspace(y1, y0, N_POINTS) # ydireciton has to be reversed for some reason
    plot_dict[k] = v * plot_dict_min_curr[k] 
    plot_dict_Ic[k] = hv.Path((np.max(image.data, axis=1) - np.min(image.data, axis=1),ydim))

In [ ]:
hv.HoloMap(plot_dict, kdims=kdims)

### Critical current

In [ ]:
%%opts Path {+axiswise +framewise}
hv.HoloMap(plot_dict_Ic, kdims=kdims).redim(x='Ic', y='B')

### Plot of current

In [ ]:
im = plot_dict[(1, 0.0, 10, 'current')]

In [ ]:
im

In [ ]:
im[:,.50:.25]


In [ ]:
data = im.items()[0][1].data

In [ ]:
hv.Curve(data[80,:])